### Import libs

In [ ]:
from Utils.Containers import *
import matplotlib.pyplot as plt
from Utils.Conversion import *
from einops import rearrange

%config IPCompleter.greedy=True
%load_ext autoreload
%autoreload 2

### Load item

In [ ]:
_path = r"path_to_dcm"
_container = SeriesContainer(verbose = True)
_item = _container.obtain_data_from_series(path_dir = _path, mode='o')[0]

print(_item['series_image_info'])
print(_item['series_tags'])

### Display image

In [ ]:
## Sum array
_image = _item['image']
_summed_2d_array = np.sum(_image, axis=2)
plt.imshow(_summed_2d_array, cmap='gray')
plt.colorbar()
plt.title('Summed 2D Array')
plt.xlabel('X dimension')
plt.ylabel('Y dimension')
plt.show()


### Generate coronal image

In [ ]:
_reconstructed_image = reconstruct_image(_item)
_resampled_image = resample_image(image = _reconstructed_image)
_resampled_item = transfer_to_numpy(_resampled_image)
_image = transfer_to_coronal(image = _resampled_item['image'], 
                   tags = _item['series_tags'])

### Plot coronal image

In [ ]:
_summed_2d_array = np.sum(_image, axis=2)
plt.imshow(_summed_2d_array.T, cmap='gray')
plt.colorbar()
plt.title('Summed 2D Array')
plt.xlabel('X dimension')
plt.ylabel('Y dimension')
plt.show()

### Show histogram

In [ ]:
plot_image_hist(_image, 1000)

### Do histogram reduction

In [ ]:
_image, _roi, min_value = histogram_reduction(image = _image,
                                              search_region = 400,
                                              reduction_factor = 50, 
                                              number_of_bins = 500,
                                              upper_bound = 1200)
print(f"ROI: {_roi}, Min Value: {min_value}")


In [ ]:
#_image = transfer_to_coronal(image = _item['image'], 
#                  tags = _item['series_tags'])

In [ ]:
## Sum array
_image = _image
_summed_2d_array = np.sum(_image, axis=2)

plt.imshow(_summed_2d_array.T, cmap='gray')
plt.colorbar()
#plt.clim(-500,500)
plt.title('Summed 2D Array')
plt.xlabel('X dimension')
plt.ylabel('Y dimension')
plt.savefig('tst1.png')
plt.show()


### ----------------------------------------------------------------------------------------------------------------------------------

# Extracting from zip -- full pipline test

### Libs

In [ ]:
import zipfile
import os
from Utils.Containers import *
import matplotlib.pyplot as plt
from Utils.Conversion import *
import shutil
%config IPCompleter.greedy=True
%load_ext autoreload
%autoreload 2

### Get path

In [ ]:
# Output dir
_dir = r"path_to_dir"
# Input dir
_path = r"path_to_dcm"

### Read zip

In [ ]:
# Extract dcms from zip
with zipfile.ZipFile(_path) as _zip_file:
    for _member in _zip_file.namelist():
        _filename = os.path.basename(_member)
        # skip directories
        if not _filename:
            continue
        if _member.endswith('.dcm'):
            _middle_dir_name = os.path.basename(os.path.dirname(_member))
            # copy file (taken from zipfile's extract)
            _source = _zip_file.open(_member)
            _target = open(os.path.join(_dir, f"{_middle_dir_name}_{_filename}"), "wb")
            with _source, _target:
                shutil.copyfileobj(_source, _target)
_case_name = os.path.splitext(os.path.basename(_path))[0]
print(_case_name)

### Get container

In [ ]:
_container = SeriesContainer(verbose = True)
_all_series = _container.obtain_data_from_series(path_dir = _dir, mode='o')

### Go through Series -> apply transforms and export

In [ ]:
# Create storage dir
os.makedirs(_case_name, exist_ok=True)
# Iterate
for _series in _all_series:
    # Create SaveDir For EachSeries
    _save_dir_name = os.path.basename(_series["main_d"]) + "__" + _series["series_ID"]
    _save_dir = os.path.join(_case_name, _save_dir_name)
    if not os.path.exists(_save_dir):
        os.makedirs(_save_dir)

    # Transfer to coronal
    # Reconstruct image
    _reconstructed_image = reconstruct_image(_series)
    # Resample image
    _resampled_image = resample_image(image = _reconstructed_image)
    # Go to numpy
    _resampled_item = transfer_to_numpy(_resampled_image)
    # Export origin image
    _image = transfer_to_coronal(image = _resampled_item['image'], 
                   tags = _series['series_tags'])
    _summed_2d_array = np.sum(_image, axis=2)
    plt.imshow(_summed_2d_array.T, cmap='gray')
    plt.axis('off')
    plt.savefig(os.path.join(_save_dir, "origin_image.png"), bbox_inches='tight', pad_inches=0, dpi = 300)


    # Do histogram reduction
    for _reduction_factor in [40, 50, 60, 70, 80, 90]:
        _reducted_image, _roi, min_value = histogram_reduction(image = _image, 
                                                      search_region = 400,
                                                      reduction_factor = _reduction_factor,
                                                      number_of_bins = 500,
                                                      upper_bound = 1200)
    
        _summed_2d_array = np.sum(_reducted_image, axis=2)
        plt.imshow(_summed_2d_array.T, cmap='gray')
        plt.axis('off')
        plt.savefig(os.path.join(_save_dir, f"{_reduction_factor}_reduction_image.png"), bbox_inches='tight', pad_inches=0, dpi = 300)
        

### ----------------------------------------------------------------------------------------------------------------------------------

## Show images in mozaik fashion

In [ ]:
from PIL import Image
import os
from multiprocessing import Process


def create_gigapixel_image(folder_paths, output_path):
    _images = []
    _max_width = 0
    _total_height = 0
    _name = f"{os.path.basename(os.path.dirname(folder_paths[0]))}_gigapixelimage.png"
    printe("Here")
    # Load images and calculate dimensions
    for _folder_path in folder_paths:
        _folder_images = []
        for _filename in os.listdir(_folder_path):
            if _filename.endswith('.jpg') or _filename.endswith('.png'):
                _image = Image.open(os.path.join(_folder_path, _filename))
                _max_width = max(_max_width, _image.width)
                _folder_images.append(_image)
                
        _total_height += _image.height
        _images.append(_folder_images)
    
    # Create gigapixel image
    _gigapixel_image = Image.new('RGB', (_max_width * len(_folder_images), _total_height))
    
    # Paste images onto gigapixel image
    _y_offset = 0
    for _folder_images in _images:
        _x_offset = 0
        for _img in _folder_images:
            _gigapixel_image.paste(_img, (_x_offset, _y_offset))
            _x_offset += _max_width
        _y_offset += _img.height
    
    # Save gigapixel image
    _gigapixel_image.save(os.path.join(output_path, _name))

# Example usage:
_main_dir = r"path_to_dir"
_output_path = "."

for _dir in os.listdir(_main_dir):
    _name = os.path.join(_main_dir, _dir)
    _folder_paths = [os.path.abspath(os.path.join(_name, _d)) for _d in os.listdir(_name)]
    _p = Process(target = create_gigapixel_image, args = (_folder_paths, _output_path,))
    _p.start()
_p.join()
#folder_paths = [os.path.abspath(os.path.join(directory, p)) for p in os.listdir(directory)]

#create_gigapixel_image(folder_paths, output_path)

In [ ]:
import zipfile
import os
from Utils.Containers import *
import matplotlib.pyplot as plt
from Utils.Conversion import *
import shutil
%config IPCompleter.greedy=True
import time
%load_ext autoreload
%autoreload 2

In [ ]:
_path_to_zip = r"C:\Users\CH258598\Desktop\Current Research\PlayingData\case-122285.zip"
_target_dir = r"C:\Users\CH258598\Desktop\Current Research\3Dto2D"
_zip_export_dir = extract_zip(input_zip_path = _path_to_zip, 
                              target_dir_path = _target_dir,
                             preserve_dir_structure = True,
                             case_number = "OMG",
                             verbose = True)


In [ ]:
_container = SeriesContainer(verbose = True)
_all_series = _container.obtain_data_from_series(path_dir = _zip_export_dir, mode='o')

In [ ]:
_container = SeriesContainer(verbose = True)
#_zip_export_dir = r"C:\Users\CH258598\Desktop\Current Research\PlayingData\case-122285 - Copy"
_all_series = _container.obtain_data_from_series(path_dir = _zip_export_dir, mode='o')

In [ ]:
start = time.time()
_container.export_data_to_directory(export_dir_path="Results", export_npy = False )
end = time.time()
print(end - start)

In [ ]:
start = time.time()
_container.export_images_to_directory("Results",
                                     resample_spacing = [1,1,1])
end = time.time()
print(end - start) 635

In [ ]:
import SimpleITK as sitk
import sys
import os

_dcm_files = ['path_to_dcm']

for _dcm in _dcm_files:
    print(_dcm)
    reader = sitk.ImageFileReader()

    reader.SetFileName(_dcm)
    #reader.LoadPrivateTagsOn()

    reader.ReadImageInformation()
    print(f"Image Size: {reader.GetSize()}")
#reader = sitk.ImageSeriesReader([_dcm])
#print("Reading Dicom directory:", sys.argv[1])


#dicom_names = reader.GetGDCMSeriesFileNames(sys.argv[1])
#reader.SetFileNames(dicom_names)

#image = reader.Execute()

#size = image.GetSize()
#print("Image size:", size[0], size[1], size[2])
